# ENPM808W Spotify Project - Experimental
In this notebook, we tried to implement a Deep Neural Network to fit the data to genres, i.e. Genre Classification. This is an experimental notebook. This did not work out for us due to the non-covergence in the model caused by the vast dimensionality of the data.

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [25]:
# Load datasets
data_by_genres = pd.read_csv('/content/drive/My Drive/Project Dataset/archive/data_by_genres.csv')
data_w_genres =  pd.read_csv('/content/drive/My Drive/Project Dataset/archive/data_w_genres.csv')

## Representing the dataset
We represent the dataset to understand how it is like

In [4]:
data_by_genres.head(10)

,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode
0,432hz,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.131000,-16.854000,0.076817,120.285667,0.221750,52.166667,5,1
1,[],0.679245,0.521473,2.298526e+05,0.401522,0.196737,0.221586,-12.679076,0.112267,112.025168,0.515920,21.682005,7,1
2,a cappella,0.621532,0.577017,1.936522e+05,0.345694,0.003799,0.127087,-12.770211,0.095324,111.813230,0.453186,43.351819,11,1
3,abstract,0.359395,0.459500,3.430185e+05,0.487000,0.791400,0.119480,-14.092000,0.043420,124.743200,0.304990,41.500000,1,1
4,abstract beats,0.353347,0.694400,2.338244e+05,0.613400,0.349403,0.102453,-6.699800,0.143453,119.398400,0.634187,58.600000,10,0
5,abstract hip hop,0.205872,0.723132,2.490951e+05,0.645461,0.002853,0.168032,-7.216007,0.250104,112.160287,0.584392,43.804971,11,1
6,accordeon,0.801333,0.626333,1.626133e+05,0.309000,0.616000,0.252667,-10.736667,0.035733,114.522000,0.543667,28.666667,2,1
7,accordion,0.486208,0.640833,1.665082e+05,0.364000,0.111317,0.186975,-14.436542,0.082775,112.722167,0.643917,15.208333,2,1
8,acid house,0.057002,0.684228,2.936093e+05,0.721419,0.320219,0.232381,-10.378311,0.058509,123.175933,0.622206,44.566667,11,1
9,acid jazz,0.155414,0.714200,2.843624e+05,0.694667,0.190613,0.106800,-10.234667,0.052020,112.243400,0.789367,44.033333,6,1


In [5]:
data_w_genres.head()

,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count,genres
0,"""Cats"" 1981 Original London Cast",0.575083,0.442750,247260.000000,0.386336,0.022717,0.287708,-14.205417,0.180675,115.983500,0.334433,38.000000,5,1,12,['show tunes']
1,"""Cats"" 1983 Broadway Cast",0.862538,0.441731,287280.000000,0.406808,0.081158,0.315215,-10.690000,0.176212,103.044154,0.268865,33.076923,5,1,26,[]
2,"""Fiddler On The Roof” Motion Picture Chorus",0.856571,0.348286,328920.000000,0.286571,0.024593,0.325786,-15.230714,0.118514,77.375857,0.354857,34.285714,0,1,7,[]
3,"""Fiddler On The Roof” Motion Picture Orchestra",0.884926,0.425074,262890.962963,0.245770,0.073587,0.275481,-15.639370,0.123200,88.667630,0.372030,34.444444,0,1,27,[]
4,"""Joseph And The Amazing Technicolor Dreamcoat""...",0.605444,0.437333,232428.111111,0.429333,0.037534,0.216111,-11.447222,0.086000,120.329667,0.458667,42.555556,11,1,9,[]


In [6]:
data_by_genres['genres'].value_counts()

greek contemporary classical    1
avant-garde metal               1
australian post-hardcore        1
dutch trance                    1
desi hip hop                    1
                               ..
israeli hip hop                 1
musikkorps                      1
hindustani instrumental         1
prepared piano                  1
chamber ensemble                1
Name: genres, Length: 2664, dtype: int64

In [7]:
# filter genres text by dropping []
df = data_by_genres.drop(data_by_genres[data_by_genres['genres'] == '[]'].index)
print(len(df['genres'].unique()))
print(len(df['genres']))

2663
2663


In [26]:
# Genre information rap and jazz
print(df[df['genres']=='rap'].value_counts())
print(df[df['genres']=='jazz'].value_counts())

genres  acousticness  danceability  duration_ms    energy    instrumentalness  liveness  loudness   speechiness  tempo      valence   popularity  key  mode
rap     0.150576      0.722337      240354.883486  0.666648  0.014343          0.20951   -7.347116  0.208261     116.14225  0.553055  52.575048   1    1       1
dtype: int64
genres  acousticness  danceability  duration_ms    energy    instrumentalness  liveness  loudness    speechiness  tempo       valence   popularity  key  mode
jazz    0.751593      0.505912      331072.141345  0.321045  0.374473          0.192582  -15.024195  0.059999     112.610828  0.483143  21.408293   5    1       1
dtype: int64


In [9]:
df.sort_values(by='popularity', ascending=False)

,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode
1118,guaracha,0.009030,0.745000,189818.000000,0.972000,0.465000,0.297000,-3.506000,0.077400,128.031000,0.556000,82.000000,7,1
512,circuit,0.009030,0.745000,189818.000000,0.972000,0.465000,0.297000,-3.506000,0.077400,128.031000,0.556000,82.000000,7,1
2141,russian dance,0.005610,0.653000,198095.000000,0.945000,0.915000,0.439000,-2.634000,0.096000,126.093000,0.326000,79.000000,5,1
34,afroswing,0.367658,0.697292,186504.958333,0.564667,0.000180,0.210796,-7.439500,0.180471,79.759917,0.701125,77.625000,11,0
2104,ritmo kombina,0.251333,0.845000,172747.333333,0.578333,0.000040,0.069533,-5.927667,0.111667,132.154000,0.505333,77.333333,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1786,norman ok indie,0.981000,0.506000,184790.000000,0.120000,0.000067,0.295000,-11.685000,0.058400,110.825000,0.456000,0.000000,4,1
1406,jewish cantorial,0.982000,0.214000,278560.000000,0.143000,0.000003,0.084700,-11.231000,0.037900,70.720000,0.053300,0.000000,2,0
1286,islamic recitation,0.988000,0.555000,184172.000000,0.304000,0.000133,0.223000,-9.738000,0.407000,84.909000,0.711000,0.000000,2,0
1635,modern free jazz,0.961000,0.308000,157889.000000,0.308000,0.824000,0.187000,-12.950000,0.039500,116.145000,0.355000,0.000000,2,1


In [10]:
len(df['genres'].unique())

2663

In [11]:
df.head()

,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode
0,432hz,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.131000,-16.854000,0.076817,120.285667,0.221750,52.166667,5,1
2,a cappella,0.621532,0.577017,1.936522e+05,0.345694,0.003799,0.127087,-12.770211,0.095324,111.813230,0.453186,43.351819,11,1
3,abstract,0.359395,0.459500,3.430185e+05,0.487000,0.791400,0.119480,-14.092000,0.043420,124.743200,0.304990,41.500000,1,1
4,abstract beats,0.353347,0.694400,2.338244e+05,0.613400,0.349403,0.102453,-6.699800,0.143453,119.398400,0.634187,58.600000,10,0
5,abstract hip hop,0.205872,0.723132,2.490951e+05,0.645461,0.002853,0.168032,-7.216007,0.250104,112.160287,0.584392,43.804971,11,1


In [12]:
in_columns = [df.columns[i] for i in range(len(df.columns))]
print(in_columns)

['genres', 'acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence', 'popularity', 'key', 'mode']


In [14]:
df_known = df.copy()

In [15]:
df_feature = df_known.drop('genres', axis=1)
df_target = df_known.drop(df_feature.columns, axis=1)
df_feature.head()
df_target.head(20)

,genres
0,432hz
2,a cappella
3,abstract
4,abstract beats
5,abstract hip hop
6,accordeon
7,accordion
8,acid house
9,acid jazz
10,acid rock


## Step 1: Pre-processing
Perform the preprocessing of the Output variable (Genre) by using One Hot Encoding

In [16]:
# Load pre-processing libraries
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

label_encode = LabelEncoder()
int_encode = label_encode.fit_transform(np.array(df_target).ravel())
Target = OneHotEncoder(sparse=False, drop='first').fit_transform(int_encode.reshape(-1,1))
print(Target)
inverted = label_encode.inverse_transform([np.argmax(Target[0, :])+1]) 
print(inverted)


[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
['a cappella']


In [27]:
# Load PyTorch library
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [18]:
# Dataset split to train and test
train_x_orig, test_x_orig, train_y_orig, test_y_orig = train_test_split(df_feature, Target, test_size=0.3, random_state=2)
train_x, train_y = train_x_orig.T, train_y_orig.T
test_x, test_y = test_x_orig.T, test_y_orig.T 
print("Train Size: X Features {}, Y Target {}".format(train_x.shape, train_y.shape))
print("Test Size: X Features {}, Y Target {}".format(test_x.shape, test_y.shape))

Train Size: X Features (13, 1864), Y Target (2662, 1864)
Test Size: X Features (13, 799), Y Target (2662, 799)


## Step 2: Implement a FCN 
implementing a Neural Network from scratch

In [32]:
class FCN(nn.Module):
  """
  Model class creates a Neural network
  param : Input Layer size, hidden layer 1 size, hidden layer 2 size, output layer size
  return : Activated output
  """
  def __init__(self, n_input, n_hidden1, n_hidden2, n_output):
    super(FCN,self).__init__()
    self.fc1 = nn.Linear(n_input, n_hidden1)
    self.fc2 = nn.Linear(n_hidden1, n_hidden2)
    self.fc3 = nn.Linear(n_hidden2, n_output)
    self.softmax = nn.Softmax(dim=1)
  
  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = self.fc3(x)
    # out = torch.softmax(x, dim=1)
    # out = self.softmax(x)
    out = torch.sigmoid(x)
    return out

In [33]:
# Setting the hyper-parameters of the model
learning_rate = 0.01
momentum = 0.9
weight_decay = 0.01
epochs = 1000

n_x = train_x.shape[0]
n_h1 = 13
n_h2 = 7
n_y = train_y.shape[0]

model = FCN(n_x, n_h1, n_h2, n_y) # Load model
loss_func = nn.MultiLabelSoftMarginLoss() #Defining the loss function: Muli Label loss
optim = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum,weight_decay = weight_decay) #Create optimizer based on 'SGD'

In [34]:
# Convert Numpy to tensor
TrainX = torch.from_numpy(np.array(train_x))
TrainY = torch.from_numpy(np.array(train_y))
TestX = torch.from_numpy(np.array(test_x))
TestY = torch.from_numpy(np.array(test_y))

## Step 3: Train Model

In [35]:
# Train Model
model.train()

training_losses = []
for epoch in range(0,epochs):
  model_output = model(TrainX.T.float())
  loss = loss_func(model_output,TrainY.T.float())
  training_losses.append(loss)

  optim.zero_grad()
  loss.backward()
  optim.step()
  if epoch%100==0:
    print("Epoch: {}, Training Loss: {}".format(epoch,loss.item()))

print("Training Loss: "+str(training_losses[-1].float()))

Epoch: 0, Training Loss: 1.0099961757659912
Epoch: 100, Training Loss: 1.0095363855361938
Epoch: 200, Training Loss: 1.0093032121658325
Epoch: 300, Training Loss: 1.0093032121658325
Epoch: 400, Training Loss: 1.009303092956543
Epoch: 500, Training Loss: 1.0037118196487427
Epoch: 600, Training Loss: 1.0027821063995361
Epoch: 700, Training Loss: 1.0025495290756226
Epoch: 800, Training Loss: 1.002549409866333
Epoch: 900, Training Loss: 1.002083659172058
Training Loss: tensor(1.0014, grad_fn=<MeanBackward0>)


In [36]:
# Evaluate model and predictions
model.eval()
predictions = model(TestX.T.float())
print(predictions)
predictions[predictions > 0.18] = 1
predictions[predictions < 0.18] = 0
print(predictions)

tensor([[1.0000e+00, 1.0000e+00, 4.1658e-10,  ..., 0.0000e+00, 1.0000e+00,
         1.0000e+00],
        [1.0000e+00, 1.0000e+00, 3.8575e-08,  ..., 0.0000e+00, 1.0000e+00,
         1.0000e+00],
        [1.0000e+00, 1.0000e+00, 1.0976e-13,  ..., 0.0000e+00, 1.0000e+00,
         1.0000e+00],
        ...,
        [1.0000e+00, 1.0000e+00, 1.7103e-08,  ..., 0.0000e+00, 1.0000e+00,
         1.0000e+00],
        [1.0000e+00, 1.0000e+00, 5.9609e-11,  ..., 0.0000e+00, 1.0000e+00,
         1.0000e+00],
        [1.0000e+00, 1.0000e+00, 9.2356e-11,  ..., 0.0000e+00, 1.0000e+00,
         1.0000e+00]], grad_fn=<SigmoidBackward>)
tensor([[1., 1., 0.,  ..., 0., 1., 1.],
        [1., 1., 0.,  ..., 0., 1., 1.],
        [1., 1., 0.,  ..., 0., 1., 1.],
        ...,
        [1., 1., 0.,  ..., 0., 1., 1.],
        [1., 1., 0.,  ..., 0., 1., 1.],
        [1., 1., 0.,  ..., 0., 1., 1.]], grad_fn=<IndexPutBackward>)


In [37]:
# Evaluation using confusion matrix
# The model overfits and does not converge
y_test = torch.max(TestY.T, 1)[1].detach().numpy()
y_pred = torch.max(predictions, 1)[1].detach().numpy()
print(y_test.shape)
print(y_pred.shape)
confusion_matrix(y_pred, y_test)

(799,)
(799,)


array([[0, 1, 1, ..., 1, 1, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [38]:
pip freeze > requirements.txt